You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article





```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 #Install
```

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guida

In [93]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [94]:
from crewai import Agent, Task, Crew

- As a LLM for your agents,will be using OpenAI's `gpt-3.5-turbo`.



In [95]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPER_API_KEY"] = ""
serper_key = os.environ.get("SERPER_API_KEY")
key = os.environ.get("OPENAI_API_KEY")
print(key[:8] + "...")  # prints first 8 chars
print("Serper key:", serper_key[:8] + "...")


sk-proj-...
Serper key: a8422508...


In [89]:
!pip install requests


In [96]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [97]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [98]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [99]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [100]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [101]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [102]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [82]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [103]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan Document:

Title: The Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence and its impact on various industries
- Mention the rapid advancements and innovations in AI technology
- Tease upcoming trends and key players in the field

Key Points:
1. Latest Trends in Artificial Intelligence:
- Natural Language Processing (NLP) and conversational AI
- AI-driven automation and robotics
- AI in healthcare and personalized medicine
- AI ethics

- Display the results of your execution as markdown in the notebook.

In [104]:
from IPython.display import Markdown
Markdown(result)

# The Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way we work and interact with technology. The rapid advancements in AI technology have paved the way for innovative solutions that were once thought to be purely science fiction. From natural language processing to automation in robotics, AI is reshaping the future of work and daily life. In this blog post, we will delve into the latest trends and key players in the field of Artificial Intelligence, highlighting the cutting-edge developments that are shaping the AI landscape.

## Latest Trends in Artificial Intelligence

### Natural Language Processing (NLP) and Conversational AI
One of the most significant trends in AI is the rise of natural language processing (NLP) and conversational AI. These technologies enable machines to understand and generate human language, leading to the development of virtual assistants like Siri and Alexa. The ability to communicate with machines in a natural and conversational manner is revolutionizing customer service, healthcare, and various other industries.

### AI-Driven Automation and Robotics
AI-driven automation and robotics are transforming industries by streamlining processes, increasing efficiency, and reducing human error. From self-driving cars to automated manufacturing systems, AI is revolutionizing the way we work and live. Businesses are increasingly turning to AI-powered solutions to optimize their operations and drive growth.

### AI in Healthcare and Personalized Medicine
AI has made significant strides in the healthcare industry, with applications ranging from medical imaging to drug discovery. Personalized medicine, powered by AI algorithms, is revolutionizing patient care by tailoring treatments to individual genetic profiles. The integration of AI in healthcare is leading to improved diagnostics, treatment outcomes, and patient experiences.

### AI Ethics and Bias Mitigation
As AI technologies become more pervasive, concerns about ethics and bias have come to the forefront. Ensuring that AI systems are developed and deployed ethically is crucial to building trust and accountability. Efforts to mitigate bias in AI algorithms and promote transparency in decision-making are essential for the responsible advancement of AI technology.

## Key Players in Artificial Intelligence

### Google's DeepMind
Google's DeepMind is a leader in AI research, known for its groundbreaking work in deep learning and reinforcement learning. The company has developed AI systems that have achieved remarkable feats, from mastering complex games to advancing healthcare research. DeepMind's contributions to the field of AI have positioned it as a key player in shaping the future of artificial intelligence.

### IBM Watson
IBM Watson is a cognitive computing platform that leverages AI to analyze and interpret vast amounts of data. Watson's capabilities span across various industries, including healthcare, finance, and retail. The platform's natural language processing and machine learning capabilities have made it a go-to solution for businesses looking to harness the power of AI for decision-making and innovation.

### Amazon Web Services (AWS) AI
Amazon Web Services (AWS) offers a suite of AI services that enable businesses to integrate AI capabilities into their applications and workflows. From speech recognition to image analysis, AWS AI services provide scalable and cost-effective solutions for organizations looking to leverage AI technologies. AWS's commitment to democratizing AI has made it a key player in the AI market.

### Microsoft AI Research
Microsoft's AI Research division is at the forefront of developing cutting-edge AI technologies, from computer vision to natural language understanding. The company's dedication to advancing AI research and collaborating with the global AI community has positioned it as a leader in the field. Microsoft's AI initiatives span across healthcare, education, and environmental sustainability, showcasing the diverse applications of AI technology.

### OpenAI
OpenAI is a research organization dedicated to developing AI technologies that benefit humanity. The organization focuses on creating safe and beneficial AI systems that have a positive impact on society. OpenAI's research spans across a wide range of domains, from robotics to reinforcement learning, with a commitment to ethical AI practices and transparency.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI research have propelled the field forward, with advancements in machine learning, neural networks, and AI ethics. Industry collaborations and partnerships have fostered innovation and accelerated the development of AI applications. From autonomous vehicles to smart cities, AI is being harnessed to solve real-world problems and drive positive change in society.

## Conclusion

Artificial Intelligence is a dynamic and rapidly evolving field that holds immense potential for shaping the future of technology and society. The latest trends in AI, from natural language processing to personalized medicine, are transforming industries and revolutionizing the way we live and work. Key players in the AI space, such as Google's DeepMind and IBM Watson, are driving innovation and pushing the boundaries of what AI can achieve. As AI continues to progress, it is essential to prioritize ethical considerations and mitigate bias to ensure the responsible development and deployment of AI technologies. Stay informed and engaged with the latest developments in AI to be at the forefront of this transformative technology revolution.

*Stay updated on the latest trends and innovations in Artificial Intelligence by subscribing to our newsletter or following AI thought leaders in the industry. Share your thoughts and experiences with AI in the comments section below.*

End of Blog Post.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [106]:
topic = "Makeuptrends in India"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Makeuptrends in India.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Makeup Trends in India

I. Introduction
- Brief overview of the makeup industry in India
- Importance of staying updated on makeup trends
- Preview of key points to be discussed

II. Latest Makeup Trends in India
A. Bold Lip Colors
- Trend towards bold lip colors such as deep reds and purples
- Influence from Bollywood celebrities and social media influencers
- Tips on how to pull off bold lip colors

B. Glowing Skin
- Emphasis on natural, glowing skin
- Popular use of highlighters and illuminatin

In [107]:
Markdown(result)

# Makeup Trends in India

## Introduction

The makeup industry in India has experienced remarkable growth, driven by a surge in demand for innovative products. Keeping up with the latest makeup trends is essential for both enthusiasts and professionals to remain competitive. In this blog post, we will delve into the current makeup trends in India, highlight key players in the industry, and discuss significant developments shaping the beauty landscape in the country.

## Latest Makeup Trends in India

### Bold Lip Colors
A standout trend in India is the rising popularity of bold lip colors, particularly deep reds and purples. Influenced by Bollywood celebrities and social media influencers, these vibrant shades have taken the makeup scene by storm. Pulling off bold lip colors requires selecting the right shade that complements your skin tone and outfit. For a striking look, pairing a bold lip with minimal eye makeup is ideal for special occasions.

### Glowing Skin
Glowing skin has become a focal point in India, with an emphasis on achieving a natural, radiant look through the use of highlighters and illuminating products. A solid skincare routine is essential in attaining glowing skin, making it crucial to invest in quality skincare products. Incorporating hydrating serums and moisturizers into your daily regimen can help you achieve the coveted dewy complexion.

### Eye Makeup
Indian trends in eye makeup lean towards dramatic looks, with graphic eyeliner and bold eyeshadow colors gaining popularity. Tutorials on social media platforms offer guidance on creating these eye-catching looks, adding a fun and playful element to your overall makeup style.

## Key Players in the Indian Makeup Industry

### Lakmé
Lakmé is a leading makeup brand in India renowned for its high-quality products and innovative collaborations. The brand frequently collaborates with top Indian designers for fashion shows, showcasing the latest makeup trends on the runway. Lakmé's introduction of innovative makeup products has solidified its position as a key player in the Indian beauty industry.

### Nykaa
Nykaa, a popular online beauty retailer in India, has made a name for itself with a range of makeup products tailored to Indian skin tones. Collaborations with influencers have been instrumental in promoting Nykaa products and reaching a broader audience. Nykaa's commitment to offering a diverse range of makeup options has positioned it as a go-to destination for beauty enthusiasts in India.

## Noteworthy News in the Indian Makeup Industry

### Launch of Sustainable Makeup Brands
India is witnessing a growing trend towards eco-friendly and sustainable makeup products, with the rise of cruelty-free and vegan brands gaining traction. Consumers are increasingly mindful of the environmental impact of their beauty choices, leading to a heightened demand for sustainable makeup options.

### Rise of Beauty Influencers
Beauty bloggers and vloggers wield significant influence in the Indian beauty industry, collaborating with makeup brands for product launches and promotions. Their honest reviews and tutorials assist consumers in making informed decisions about their makeup purchases. The surge of beauty influencers has revolutionized how brands market their products, prioritizing authentic endorsements to connect with consumers.

## Target Audience Analysis

The target audience for makeup trends in India comprises young women aged 18-35 who are passionate about makeup and beauty. These individuals are eager to stay abreast of the latest trends, seek makeup application tips and tutorials, and are constantly on the lookout for new products to enhance their collection. By catering to this demographic, makeup brands and influencers can effectively engage with their target audience and foster brand loyalty.

In conclusion, the makeup industry in India is flourishing, offering a diverse array of trends and products to meet consumer needs. Keeping up with the latest makeup trends enables individuals to elevate their beauty routines and experiment with fresh looks. Whether you are a makeup enthusiast or a beauty professional, embracing these trends can aid in staying ahead of the curve and expressing creativity through makeup.

*Stay informed about the latest makeup trends in India by following our blog for additional tips, tutorials, and industry news.*

---
By adhering to the content plan provided, this blog post aims to provide valuable insights into the makeup trends in India, engage readers with relevant content, and bolster traffic through SEO optimization.

In [108]:
topic = "Online plant selling webistes in India"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Online plant selling webistes in India.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: The Growing Trend of Online Plant Selling Websites in India

Introduction:
- Brief overview of the rise in popularity of online plant selling websites in India
- Introduction of key players in the market
- Mention of noteworthy news related to the industry

Key Points:
1. Latest Trends in Online Plant Selling Websites in India:
- Increased demand for indoor plants due to work from home culture
- Emphasis on sustainable and eco-friendly gardening practices
- Intro

In [109]:
Markdown(result)

# The Growing Trend of Online Plant Selling Websites in India

## Introduction

In recent years, online plant selling websites have seen a significant rise in popularity in India. This surge can be attributed to various factors such as the increased demand for indoor plants due to the work from home culture, the emphasis on sustainable and eco-friendly gardening practices, and the introduction of virtual plant care services for customers. Key players in the market, including Nurserylive, Ugaoo, and MyBageecha, have been at the forefront of this trend, offering a wide range of plant varieties and services to cater to the growing needs of urban dwellers, plant enthusiasts, and eco-conscious individuals. Noteworthy news related to the industry, such as recent collaborations between online plant sellers and environmental organizations, the expansion of product offerings to include rare and exotic plant species, and the impact of COVID-19 on the growth of online plant selling websites in India, further highlight the dynamism of this sector.

## Latest Trends in Online Plant Selling Websites in India

One of the latest trends in the online plant selling industry in India is the increased demand for indoor plants. With more people spending time indoors due to the work from home culture, there has been a growing interest in incorporating greenery into living spaces to create a more vibrant and healthy environment. Additionally, there is a noticeable emphasis on sustainable and eco-friendly gardening practices, with online platforms promoting the use of organic fertilizers, biodegradable pots, and water-saving techniques to reduce the environmental impact of gardening. Moreover, the introduction of virtual plant care services has been a game-changer for customers, providing them with expert advice on plant care and maintenance right at their fingertips.

## Key Players in the Market

Nurserylive, Ugaoo, and MyBageecha are among the key players in the online plant selling market in India, each offering a unique set of services and products to cater to the diverse needs of customers. Nurserylive, known for its vast selection of plants and gardening accessories, has garnered a loyal customer base for its quality products and reliable delivery services. Ugaoo, on the other hand, stands out for its focus on sustainable gardening practices, offering customers a wide range of eco-friendly products and resources to help them create a green and thriving garden. MyBageecha, with its emphasis on rare and exotic plant species, appeals to plant enthusiasts looking for unique additions to their collections. Customer reviews and ratings play a crucial role in establishing the credibility of these platforms, helping potential buyers make informed decisions based on the experiences of others.

## Noteworthy News in the Industry

Recent collaborations between online plant sellers and environmental organizations have brought attention to the importance of sustainability and conservation in the gardening industry. By partnering with conservation groups and participating in tree-planting initiatives, online plant selling websites are not only promoting environmental awareness but also contributing to the preservation of natural habitats. Furthermore, the expansion of product offerings to include rare and exotic plant species has opened up new possibilities for plant lovers, allowing them to explore a diverse range of plants from around the world. The impact of COVID-19 on the growth of online plant selling websites in India cannot be overlooked, as the pandemic has accelerated the shift towards online shopping and highlighted the convenience and safety of purchasing plants from the comfort of one's home.

In conclusion, the online plant selling industry in India is experiencing a period of rapid growth and innovation, driven by the changing needs and preferences of consumers. With a focus on sustainability, quality, and customer satisfaction, online platforms are revolutionizing the way people buy and care for plants, making gardening more accessible and enjoyable for everyone. As we navigate the digital landscape of plant shopping, it is essential to support reputable sellers, educate ourselves on plant care practices, and embrace the beauty of nature in our homes and gardens.

As a content writer, it is essential to provide valuable insights and information to readers looking to explore the world of online plant selling websites in India. By incorporating relevant SEO keywords and resources, addressing the interests and pain points of the target audience, and offering a compelling call to action, this blog post aims to engage and inform readers about the latest trends, key players, and noteworthy news in the industry. Let's continue to nurture our love for plants and create green havens in our urban spaces with the help of online plant selling websites in India.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).